In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

os.chdir('/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/')

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128
IMAGE_SIZE = [96, 96]

## Load the data

In [7]:
import tensorflow as tf

FILENAMES1 = tf.io.gfile.glob("group1/" + "*.tfrecords" )
FILENAMES2= tf.io.gfile.glob("group2/" + "*.tfrecords" )
FILENAMES3= tf.io.gfile.glob("extrautkface/" + "*.tfrecords" )
FILENAMES4= tf.io.gfile.glob("group3/" + "*.tfrecords" )
FILENAMES5= tf.io.gfile.glob("group4/" + "*.tfrecords" )
FILENAMES6= tf.io.gfile.glob("group5/" + "*.tfrecords" )

FILENAMES=FILENAMES3+ FILENAMES1+FILENAMES2 + FILENAMES4 +FILENAMES5 +FILENAMES6

split_ind = int(0.7 * len(FILENAMES))
TRAINING_FILENAMES, VALID_FILENAMES = FILENAMES[:split_ind], FILENAMES[split_ind:len(FILENAMES)-1]

TEST_FILENAMES = FILENAMES[len(FILENAMES)-1:]

print("Train TFRecord Files:", len(TRAINING_FILENAMES))
print("Validation TFRecord Files:", len(VALID_FILENAMES))
print("Test TFRecord Files:", len(TEST_FILENAMES))

Train TFRecord Files: 70
Validation TFRecord Files: 30
Test TFRecord Files: 1


### Decoding the data 

The images have to be converted to tensors so that it will be a valid input in our model.
As images utilize an RBG scale, we specify 3 channels.



In [12]:
import keras.backend as K
import numpy as np 


def decode_image(image_raw):
    image_shape = tf.stack([96, 96, 3])
    image = tf.io.decode_raw(image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, image_shape)
    
    return image


def read_tfrecord(example, labeled):
    tf.compat.v1.enable_eager_execution()
    tfrecord_format = (
        {
            "label_regr": tf.io.FixedLenFeature([],tf.float32),
            "label_class": tf.io.FixedLenFeature([101],tf.int64),
            "label_order" : tf.io.FixedLenFeature([101],tf.int64),
            "image" : tf.io.FixedLenFeature([],tf.string),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled: 
        label = example["label_class"]
        return image, label
    return image
 

In [11]:
tf.version.VERSION

'2.4.0'

### Functions to load Data

Create functions to load data

In [22]:
from functools import partial

def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset


We define the following function to get our different datasets.

In [23]:

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset


In [17]:
from keras.layers import Flatten, Dense, Input, Dropout
from tensorflow.keras import layers, models, Model, optimizers
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

def mobilenet_96_build(input_shape=(96,96,3), num_classes=101, weights="imagenet"):
    global lr_reduce,checkpoint
    print("Building mobilenet 96", input_shape, "- num_classes", num_classes, "- weights", weights)
    m1 = keras.applications.mobilenet_v2.MobileNetV2(input_shape, 0.75, include_top=True, weights=weights)
    features = m1.layers[-2].output
    x = keras.layers.Dense(num_classes, activation='softmax', use_bias=True, name='Logits')(features)
    model = keras.models.Model(m1.input, x)
    lr_reduce = ReduceLROnPlateau(monitor='val_mae', factor=0.6, patience=5, verbose=1, mode='auto', min_lr=5e-5)
    for l in model.layers: l.trainable = True
    return model, features

In [15]:
from tensorflow import keras

In [18]:
initial_model,features=mobilenet_96_build((96,96,3), 101)

Building mobilenet 96 (96, 96, 3) - num_classes 101 - weights imagenet


### Loading Data

In [ ]:
from google.colab.patches import cv2_imshow


#Function to show image with relative label One-Hot Encoding
def show_batch(image_batch, label_batch):
    count=0
    for n in range(0,len(image_batch)):
        cv2_imshow(image_batch[n])
        label=label_batch[n]
        print("Label:" + str(label))
        count=count+1
    print(count) 

train_dataset = get_dataset(TRAINING_FILENAMES)
valid_dataset = get_dataset(VALID_FILENAMES)
test_dataset = get_dataset(TEST_FILENAMES, labeled=False)

count=0
for image_batch,label_batch in iter(train_dataset):
  count=count+len(image_batch)
  print(str(count) + " di 1423313")
print("Number of images for training is: " + str(count))

In [ ]:
# Show image and label of a single batch in training set BATCH_SIZE=128
image_batch, label_batch = next(iter(train_dataset))
show_batch(image_batch.numpy(), label_batch.numpy())

## Building our model

In [27]:
# CUSTOM METRIC

import keras.backend as K
from tensorflow.keras.metrics import mean_absolute_error

def mae(y_true, y_pred): 
  y_true = K.cast(y_true, y_pred.dtype)

  ages_true = tf.map_fn(lambda true: K.argmax(true), y_true, dtype=tf.int64)
  ages_pred = tf.map_fn(lambda pred: K.argmax(pred), y_pred, dtype=tf.int64)

  return mean_absolute_error(ages_true, ages_pred)

In [28]:
# model compiling
from keras.regularizers import l2 
from keras.optimizers import SGD

model=initial_model
weight_decay = 0.005 
for layer in model.layers:
    if isinstance(layer, keras.layers.Conv2D) and not isinstance(layer, keras.layers.DepthwiseConv2D) or isinstance(
            layer, keras.layers.Dense):
        layer.add_loss(lambda: l2(weight_decay)(layer.kernel))
    if hasattr(layer, 'bias_regularizer') and layer.use_bias:
        layer.add_loss(lambda: l2(weight_decay)(layer.bias))
optimizer = SGD(momentum=0.9)

loss = keras.losses.categorical_crossentropy 
accuracy_metrics = [keras.metrics.categorical_accuracy,mae] 
model.compile(loss=loss, optimizer=optimizer, metrics=accuracy_metrics)

In [29]:
def step_decay_schedule(initial_lr, decay_factor, step_size):
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch / step_size))

    return keras.callbacks.LearningRateScheduler(schedule, verbose=1)

In [30]:
lr_sched = step_decay_schedule(initial_lr=0.005,decay_factor=0.2, step_size=20)
monitor = 'val_mae' 
checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Final_Artificial_Vision/CNN/mobileNet_70TFRecords.h5', verbose=1, save_best_only=True, monitor=monitor)
callbacks_list = [lr_sched, checkpoint]#, tbCallBack]

In [ ]:
model.summary()

## Train the model

In [ ]:

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=valid_dataset,
    callbacks=callbacks_list,
)